<a href="https://colab.research.google.com/github/RoyElkabetz/Tensor-Networks-Simple-Update/blob/main/Triangular_2d_lattice_BLBQ_Spin_1_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## uncomment only if running from google.colab
# clone the git reposetory
!git clone https://github.com/RoyElkabetz/Tensor-Networks-Simple-Update
# add path to .py files for import
import sys
sys.path.insert(1, "/content/Tensor-Networks-Simple-Update/src")

Cloning into 'Tensor-Networks-Simple-Update'...
remote: Enumerating objects: 412, done.
remote: Counting objects: 100% (412/412), done.
remote: Compressing objects: 100% (345/345), done.
remote: Total 412 (delta 237), reused 147 (delta 62), pack-reused 0
Receiving objects: 100% (412/412), 1.09 MiB | 14.92 MiB/s, done.
Resolving deltas: 100% (237/237), done.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from TensorNetwork import TensorNetwork
import SimpleUpdate as su
import structure_matrix_generator as stmg
from tqdm.auto import tqdm


np.random.seed(216)

plt.rcParams.update({'font.size': 16, 
                     "figure.facecolor": 'white', 
                     "axes.facecolor": 'white', 
                     "savefig.facecolor": 'white', 
                     'savefig.edgecolor' : 'white', 
                     'figure.edgecolor' : 'white'})

# Spin-1 Simulation of a Bilinear-Biquadratic Heisenberg model on a triangular $2D$ lattice 

In [3]:
# The Tensor Network structure matrix
triangle_structure_matrix = stmg.infinite_structure_matrix_by_name('triangular')
print('The triangular 2D lattice structure matrix:')
print(triangle_structure_matrix)

The triangular 2D lattice structure matrix:
[[1 2 3 4 5 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 2 3 4 5 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 2 0 0 0 0 3 4 5 6 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 2 0 0 0 3 4 5 6 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 2 0 0 0 0 0 0 0 3 0 0 0 4 5 6 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 2 0 0 0 3 0 0 4 0 0 5 6 0 0 0]
 [0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 0 0 3 0 0 0 0 4 0 5 6 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 2 0 0 0 0 3 0 4 0 0 0 5 0 6]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 3 0 4 0 5 6]]


In [4]:
tri_tn = TensorNetwork(structure_matrix=triangle_structure_matrix, 
                       virtual_size=2)

In [8]:
# spin-1 operators
sx = np.array([[0., 1, 0],
               [1, 0, 1],
               [0, 1, 0]]) / np.sqrt(2)
sy = np.array([[0., -1j, 0],
               [1j, 0, -1j],
               [0, 1j, 0]]) / np.sqrt(2)
sz = np.array([[1., 0, 0],
               [0, 0, 0],
               [0, 0, -1]])

s_i = [sx, sy, sz]
s_j = [sx, sy, sz]
s_k = []

# get the spin-spin interaction term
interaction_term = np.zeros((np.power(sx.shape[0], 2), np.power(sx.shape[0], 2)), dtype=np.complex)
for s in s_i:
    interaction_term += np.kron(s, s)

# compute a list of hamiltonians for all angles
theta = np.linspace(0, 2 * np.pi, 20)
hamiltonian = []
for angle in theta:
    hamiltonian.append(np.cos(angle) * interaction_term + np.sin(angle) * np.power(interaction_term, 2))

The BLBQ Hamiltonian is given by

\begin{align}
    H = \cos{\left(\theta\right)}\sum_{<i, j>} \mathbf{S}_i\cdot \mathbf{S}_j + \sin{\left(\theta\right)}\sum_{<i, j>} \left(\mathbf{S}_i\cdot \mathbf{S}_j\right)^2
\end{align}

In [9]:
# Simple-Update parameters
d_max_ising = [2]
error = 1e-6
max_iterations = 200
energy = []
j_ij = [-1.] * len(tri_tn.weights)
dts = [0.1, 0.01, 0.001, 0.0001, 0.00001]

# Run
for d_max in d_max_ising:
    for i, angle in enumerate(theta):
        tri_tn = TensorNetwork(structure_matrix=triangle_structure_matrix, 
                          virtual_size=2)
        tri_su = su.SimpleUpdate(tensor_network=tri_tn, 
                                  dts=dts, 
                                  j_ij=j_ij, 
                                  h_k=0, 
                                  s_i=s_i, 
                                  s_j=s_j, 
                                  s_k=s_k, 
                                  d_max=d_max, 
                                  max_iterations=max_iterations, 
                                  convergence_error=error, 
                                  log_energy=False,
                                  print_process=False,
                                  hamiltonian=hamiltonian[i])
        tri_su.run()
        energy = tri_su.energy_per_site()
        print('| D max: {:3d} | Theta: {:2.6f} | Energy: {:3.10f} |'
        .format(d_max, angle, energy))
        tri_su.append(energy)
    

ValueError: ignored

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(17, 6))
ax[0].plot(-h_k, ising_energy, '--o', label=r'$\epsilon_0$', color='tab:red', 
           markerfacecolor='none', linewidth=1, markersize=10)
ax[0].set_xlabel(r'h')
ax[0].set_ylabel(r'$\epsilon_0$')
ax[0].set_title('$2D$ Ising model ground-state energy per-site')
ax[0].tick_params(direction='in', axis='both')
ax[0].grid()
ax[0].legend()

ax[1].plot(-h_k, np.abs(np.array(ising_z_magnetizaton)), '--o', label=r'$M_z$', 
           color='tab:blue', markerfacecolor='none', linewidth=1, markersize=10)
ax[1].plot(-h_k, ising_x_magnetizaton, '--o', label=r'$M_x$', color='tab:green', 
           markerfacecolor='none', linewidth=1, markersize=10)
ax[1].set_xlabel(r'h')
ax[1].set_ylabel(r'$M$')
ax[1].set_title(r'$2D$ Ising model $\hat{z}, \hat{x}$ magnetizations per-site')
ax[1].tick_params(direction='in', axis='both')
ax[1].grid()
ax[1].legend()